## Data we definitely want to include:
### Outdoor polygons: OSM parks and pitches (indoor = No), Parkserve local parks and recreation.
* We selected five random cities to get
1. Bellingham, Washington — ~93,000
2. Sandpoint, Idaho — ~9,000
3. Ithaca, New York — ~32,000
4. Santa Cruz, California — ~63,000
5. Cedar Rapids, Iowa — ~138,000

In [1]:
import os
import requests
import geopandas as gpd
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
import osmnx as ox
from shapely.geometry import shape


# --- CONFIG ---
PLACE_BASE_URL = "https://www2.census.gov/geo/tiger/TIGER2025/PLACE/tl_2025_{statefp}_place.zip"
CACHE_DIR = "cache/place"
os.makedirs(CACHE_DIR, exist_ok=True)

STATE_FIPS = {
    "01": "Alabama", "02": "Alaska", "04": "Arizona", "05": "Arkansas",
    "06": "California", "08": "Colorado", "09": "Connecticut", "10": "Delaware",
    "12": "Florida", "13": "Georgia", "16": "Idaho", "17": "Illinois",
    "18": "Indiana", "19": "Iowa", "20": "Kansas", "21": "Kentucky",
    "22": "Louisiana", "23": "Maine", "24": "Maryland", "25": "Massachusetts",
    "26": "Michigan", "27": "Minnesota", "28": "Mississippi", "29": "Missouri",
    "30": "Montana", "31": "Nebraska", "32": "Nevada", "33": "New Hampshire",
    "34": "New Jersey", "35": "New Mexico", "36": "New York", "37": "North Carolina",
    "38": "North Dakota", "39": "Ohio", "40": "Oklahoma", "41": "Oregon",
    "42": "Pennsylvania", "44": "Rhode Island", "45": "South Carolina",
    "46": "South Dakota", "47": "Tennessee", "48": "Texas", "49": "Utah",
    "50": "Vermont", "51": "Virginia", "53": "Washington", "54": "West Virginia",
    "55": "Wisconsin", "56": "Wyoming"
}

def load_state_places(statefp):
    """Load (or download + cache) a state's PLACE shapefile as GeoDataFrame."""
    zip_path = os.path.join(CACHE_DIR, f"tl_2025_{statefp}_place.zip")

    # Download if not already cached
    if not os.path.exists(zip_path):
        url = PLACE_BASE_URL.format(statefp=statefp)
        print(f"Downloading {STATE_FIPS[statefp]} PLACE file...")
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(zip_path, "wb") as f:
            f.write(r.content)
    else:
        print(f"Using cached file for {STATE_FIPS[statefp]}")

    # Load from local zip
    return gpd.read_file(f"zip://{zip_path}")

 
def get_city_boundary(city_name, state):
    """
    Find and return the boundary GeoDataFrame for a given city within a known U.S. state.
    `state` can be the full state name (e.g., "Colorado") or FIPS (e.g., "08").
    """
    city_name = city_name.lower()

    # Resolve statefp from either state name or FIPS code
    state_lookup = {v.lower(): k for k, v in STATE_FIPS.items()}
    if state.isdigit():
        statefp = state.zfill(2)
        state_name = STATE_FIPS.get(statefp, f"FIPS {statefp}")
    else:
        statefp = state_lookup.get(state.lower())
        if not statefp:
            raise ValueError(f"Unknown state: {state}")
        state_name = state.title()

    # Load the state’s PLACE shapefile (cached if available)
    gdf = load_state_places(statefp)

    # Case-insensitive match for city name
    city_rows = gdf[gdf["NAME"].str.lower() == city_name]

    if city_rows.empty:
        raise ValueError(f"City '{city_name.title()}' not found in {state_name}.")

    city_rows = city_rows.copy()
    city_rows["state_name"] = state_name
    return city_rows

def load_trails(city, state):
    import numpy as np

    boundary_gdf = get_city_boundary(city, state)
    boundary = boundary_gdf.geometry.iloc[0]

    # Step 1 — Broad pull of candidate hiking trails
    tags = {
        "highway": ["path"],      # path only, not footway or track
    }

    trails = ox.features_from_polygon(boundary, tags=tags)
    print("Raw path features:", len(trails))

    # Clean null geometries
    trails = trails[trails.geometry.notnull()].copy()

    # Convert polygons → boundary lines
    trails["geometry"] = trails["geometry"].apply(
        lambda g: g.boundary if g.geom_type in ["Polygon", "MultiPolygon"] else g
    )

    # Keep linework
    trails = trails[trails.geometry.type.isin(["LineString", "MultiLineString"])]
    print("Line geometries:", len(trails))

    # Step 2 — Natural-surface hiking filter
    natural_surfaces = {
        "dirt", "ground", "earth", "grass", "gravel", "fine_gravel",
        "woodchips", "mud", "rock", "sand", "compacted", "unpaved"
    }

    def is_natural_surface(val):
        if isinstance(val, list):
            val = val[0]
        if val is None:
            return False
        return str(val).lower() in natural_surfaces

    # surfaces allowed OR missing if other hiking cues exist
    surface_mask = trails["surface"].apply(is_natural_surface) | trails["surface"].isna()

    # Step 3 — Exclude paved / urban paths explicitly
    paved_surfaces = {"asphalt", "concrete", "paved", "cement", "paving_stones"}

    def is_paved(val):
        if isinstance(val, list):
            val = val[0]
        if val is None:
            return False
        return str(val).lower() in paved_surfaces

    paved_mask = trails["surface"].apply(is_paved)

    # Step 4 — foot allowed OR unspecified  
    foot_mask = trails["foot"].isin(["yes", "designated"]) | trails["foot"].isna()

    # Step 5 — bicycle not allowed  
    bike_mask = trails["bicycle"].isin(["no"]) | trails["bicycle"].isna()

    # Step 6 — bonus “hiking-specific” tags
    hiking_cues = (trails["sac_scale"].notnull()) | (trails["trail_visibility"].notnull())

    # Final mask: must be natural-surface AND foot allowed AND not paved
    final_mask = (
        surface_mask &
        (~paved_mask) &
        bike_mask &
        foot_mask
    ) | hiking_cues

    trails = trails[final_mask].copy()
    print("Filtered hiking trails:", len(trails))

    # Step 7 — Add trail classification
    def classify(row):
        if pd.notnull(row.get("sac_scale")):
            return row["sac_scale"]               # true hiking scale
        if is_natural_surface(row.get("surface")):
            return "natural_surface"
        if row.get("foot") in ["yes", "designated"]:
            return "foot_designated"
        return "hiking_trail"

    trails["trail_type"] = trails.apply(classify, axis=1)

    # Step 8 — Clip to city boundary
    if trails.crs != boundary_gdf.crs:
        trails = trails.to_crs(boundary_gdf.crs)

    trails = gpd.clip(trails, boundary).reset_index(drop=True)

    return trails





KeyboardInterrupt: 

### Pull OSM data

In [ ]:
# Get OSM data
tags = {
    "leisure": ["park", "pitch", "sports_centre"]
}
def load_osm(city, state):
    city_geom = get_city_boundary(city, state).geometry.iloc[0]
    
    # Download OSM features that *intersect* the city boundary
    gdf_osm = ox.features_from_polygon(city_geom, tags=tags)
    
    # Ensure correct CRS
    gdf_osm = gdf_osm.to_crs("EPSG:4326")

    gdf_osm = gdf_osm[gdf_osm.geometry.type.isin(["Polygon", "MultiPolygon"])]
    # Clip to city boundary (this is the key step!)
    gdf_osm_clipped = gdf_osm.clip(city_geom)
    #only get polygons
    return gdf_osm_clipped

### Pull ParkServe data

In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import shape
import json

PARKSERVE_URL = (
    "https://server7.tplgis.org/arcgis7/rest/services/"
    "ParkServe/ParkServe_ProdNew/MapServer/2/query"
)
equal_area_crs = "EPSG:5070"  # US Albers Equal Area — ideal for area in meters²


def get_city_parks_parkserve(city_name, state_name, min_acres=0.1):
    """
    Fetch ParkServe parks that intersect a SINGLE city and return in EPSG:4326.
    
    Args:
        city_name (str): Name of the city (case-insensitive)
        state_name (str): Full state name (e.g., "Washington") or FIPS code
        min_acres (float): Minimum park size in acres to keep

    Returns:
        GeoDataFrame (EPSG:4326) ready for Folium plotting
    """
    # --- 1. Get city boundary ---
    city_row = get_city_boundary(city_name, state_name).iloc[0]  # Use your existing function
    city_geom = city_row.geometry
    city_name = city_row["NAME"]

    # Build bounding envelope for ArcGIS query
    bounds = city_geom.bounds
    envelope = {
        "xmin": bounds[0],
        "ymin": bounds[1],
        "xmax": bounds[2],
        "ymax": bounds[3],
        "spatialReference": {"wkid": 4326}
    }

    # --- 2. Fetch ParkServe features ---
    features = []
    offset = 0
    while True:
        params = {
            "where": "(park_designation = 'LP' OR park_designation = 'LREC')",
            "outFields": "*",
            "f": "geojson",
            "geometry": json.dumps(envelope),
            "geometryType": "esriGeometryEnvelope",
            "spatialRel": "esriSpatialRelIntersects",
            "resultOffset": offset,
            "resultRecordCount": 1000,
            "returnGeometry": "true",
            "outSR": 5070,  # original CRS in meters
        }
        r = requests.get(PARKSERVE_URL, params=params, timeout=60)
        r.raise_for_status()
        page = r.json().get("features", [])
        if not page: break
        features.extend(page)
        if len(page) < 1000: break
        offset += 1000

    if not features:
        print(f"No parks found for {city_name}")
        return gpd.GeoDataFrame(columns=["park_name","park_designation","geometry"], crs="EPSG:4326")

    # --- 3. Convert features to GeoDataFrame ---
    parks = gpd.GeoDataFrame(
        [f.get("attributes") or f.get("properties") for f in features if f.get("geometry")],
        geometry=[shape(f["geometry"]) for f in features if f.get("geometry")],
        crs="EPSG:5070"  # original CRS from ParkServe
    )

    # Fix invalid geometries
    parks["geometry"] = parks.geometry.buffer(0)
    parks = parks[~parks.geometry.is_empty]
    # --- 4. Clip parks to city boundary ---
    parks = parks.to_crs("EPSG:4326")
    city_gdf = gpd.GeoDataFrame(geometry=[city_geom.buffer(0)], crs="EPSG:4326")
    parks_clipped = gpd.overlay(parks, city_gdf, how="intersection")
    
    # --- 5. Filter by minimum size ---
    parks_clipped = parks_clipped.to_crs(equal_area_crs)
    parks_clipped["area_acres"] = parks_clipped.geometry.area / 4046.86
    parks_clipped = parks_clipped[parks_clipped["area_acres"] >= min_acres]
    parks_clipped = parks_clipped.to_crs("EPSG:4326")  # ready for Folium


    # --- 6. Reproject to EPSG:4326 for Folium ---
    parks_clipped = parks_clipped.to_crs("EPSG:4326")

    print(f"{city_name}: {len(parks)} parks before clip, {len(parks_clipped)} after clip")
    return parks_clipped


## Visualize data
- To Do: decide wheteher to filter data by minimum size. Some OSM/ParkServe data is very small, and likely not an intential stop. However, it would have a low impact on the final results and we would not have to justify our size cutoff if we left it in.

- #### To run, just change the city and state variables

In [ ]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon

city = 'Missoula'
state = 'Montana'

city_geom = get_city_boundary(city,state)
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add ParkServe polygons (green)
gdf = get_city_parks_parkserve(city, state)
folium.GeoJson(
    gdf,
    name="ParkServe",
    style_function=lambda x: {"color": "green", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=["ParkName"] if "ParkName" in gdf.columns else [])
).add_to(m)

# Add OSM polygons (gray)
gdf_osm = load_osm(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "gray", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=["osm_category"] if "osm_category" in gdf_osm.columns else [])
).add_to(m)

# Add OSM trails (orange lines)
gdf_trails = load_trails(city, state)

folium.GeoJson(
    gdf_trails,
    name="Trails",
    style_function=lambda x: {"color": "orange", "weight": 2, "opacity": 0.9},
    tooltip=folium.GeoJsonTooltip(
        fields=["trail_type"] if "trail_type" in gdf_trails.columns else []
    )
).add_to(m)

m.save(f"{city}_park_overlap.html")
m

In [ ]:
gdf

### Indoor polygons: OSM sports halls, fitness centres. 

- Average square footage of a fitness studio: https://member.afsfitness.com/content/fitness-studio-fact-sheet
- 3,813 ft. --> 61*61 ft

In [ ]:
import osmnx as ox
import geopandas as gpd
from shapely.geometry import box
import pandas as pd

def load_osm_indoor(city, state):
    """
    Fetch OSM gyms and create ≤61×61 ft indoor zones within building footprints
    for point-only gyms, clipped by other business footprints.
    """
    # --- 1. Get city boundary ---
    
    city_geom = get_city_boundary(city, state)
    if city_geom.crs is None or city_geom.crs.to_epsg() != 4326:
        city_geom = city_geom.to_crs("EPSG:4326")
    city_geom = city_geom.geometry.iloc[0]

    # --- 2. Define OSM tags ---
    gym_tags = {"leisure": ["fitness_centre", "sports_hall"]}
    bldg_tags = {"building": True}
    business_tags = {
        "amenity": True, "shop": True, "office": True,
        "craft": True, "tourism": True, "leisure": True
    }

    # --- 3. Download all OSM features ---
    gdf_gym = ox.features_from_polygon(city_geom, tags=gym_tags)
    gdf_bldgs = ox.features_from_polygon(city_geom, tags=bldg_tags)
    gdf_biz = ox.features_from_polygon(city_geom, tags=business_tags)

    # --- 4. Separate gyms ---
    gdf_gym_points = gdf_gym[gdf_gym.geometry.geom_type == "Point"].copy()
    gdf_gym_polys = gdf_gym[gdf_gym.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()

    # --- 5. Project everything once (for meter units) ---
    crs_m = "EPSG:3857"
    gdf_gym_points = gdf_gym_points.to_crs(crs_m)
    gdf_gym_polys = gdf_gym_polys.to_crs(crs_m)
    gdf_bldgs = gdf_bldgs.to_crs(crs_m)
    gdf_biz = gdf_biz.to_crs(crs_m)

    gdf_bldgs = gdf_bldgs.reset_index(drop=True)

    # --- 6. Spatial joins: assign buildings ---
    gyms_in_bldg = gpd.sjoin(
    gdf_gym_points,   # <— no column filtering
    gdf_bldgs[["geometry"]],
    how="inner",
    predicate="within"
)
    gyms_in_bldg.rename(columns={"index_right": "building_id"}, inplace=True)

    biz_in_bldg = gpd.sjoin(gdf_biz, gdf_bldgs[["geometry"]], how="inner", predicate="within")

    # Handle dynamic naming of the building index column
    join_col = None
    for col in biz_in_bldg.columns:
        if col.startswith("index_") or col.endswith("_right"):
            join_col = col
            break
    
    if join_col is None:
        raise KeyError("Could not find building index column in business join result")
    
    # Group businesses by building ID
    biz_in_bldg.rename(columns={join_col: "building_id"}, inplace=True)
    biz_grouped = biz_in_bldg.groupby("building_id")

    # --- 7. Create indoor zones for point-based gyms ---
    zones = []
    side_ft = 150
    side_m = side_ft * .3048
    half = side_m / 2
    
    for idx, row in gyms_in_bldg.iterrows():
        pt = row.geometry
        bldg_geom = gdf_bldgs.loc[row["building_id"]].geometry
    
        # Create 61x61 ft square, clipped to building
        x, y = pt.x, pt.y
        square = box(x - half, y - half, x + half, y + half)
        zone = square.intersection(bldg_geom)
    
        # Clip by nearby businesses in same building
        if row["building_id"] in biz_grouped.groups:
            biz_points = biz_grouped.get_group(row["building_id"]).geometry
            if not biz_points.empty:
                min_dist = biz_points.distance(pt).min()
                if pd.notna(min_dist) and min_dist > 0:
                    half_dist = min(min_dist / 2.0, half)
                    zone = zone.intersection(pt.buffer(half_dist)).intersection(bldg_geom)
    
        # Preserve attributes from the original point
        zones.append({
            "geometry": zone,
            "gym_name": row.get("name", None),
            "leisure": row.get("leisure", None),
            "amenity": row.get("amenity", None),
            "building_id": row["building_id"]
        })
    
    gdf_zones = gpd.GeoDataFrame(zones, crs=crs_m)


    # --- 8. Combine with polygon gyms (already defined interiors) ---
    all_gyms = pd.concat([gdf_zones, gdf_gym_polys], ignore_index=True)
    all_gyms = all_gyms.set_crs(crs_m, allow_override=True).to_crs("EPSG:4326")

    # --- 9. Clip to city boundary ---
    all_gyms = all_gyms.clip(city_geom)

    return all_gyms


### To Run, just change the city and state variables

In [ ]:
city = 'Bellingham'
state = 'Washington'

city_geom = get_city_boundary(city,state)
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add OSM polygons (gray)
gdf_osm = load_osm_indoor(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=[c for c in ["gym_name", "leisure","osm_category"] if c in gdf_osm.columns])
).add_to(m)

# Add layer control (toggle layers)
folium.LayerControl().add_to(m)

m.save(f"{city}_indoor.html")
m

## Possible Additions: 

Trailheads and indoor play areas

Suggestion: Do not include. There are only 2,000 tagged indoor play areas in the entire world, and over 35,000 places in the US. It is not a large amount of data points to include, especially with the likely small area and focus on children (who don't typically carry phones while playing). Hiking is somewhat more feasible, however, they would need to be grabbed by county instead of city, and they are unevently distributed compared to trails. Moreover, the number of trails in the county depends a lot more on geography compared to city level parks.

In [ ]:
# Get OSM data
# --- 2. Define OSM tags ---
recreation_tags = {
    "leisure": [
        "indoor_play",
        "trailhead"
    ],
    "information": ["trailhead"],
    "highway": ["trailhead"]
}

def load_osm_uncertain(city, state):
    city_geom = get_city_boundary(city, state).geometry.iloc[0]
    
    # Download OSM features that *intersect* the city boundary
    gdf_osm = ox.features_from(city+","+state, tags=recreation_tags)
    
    # Ensure correct CRS
    gdf_osm = gdf_osm.to_crs("EPSG:4326")
    #gdf_osm = gdf_osm[gdf_osm.geometry.type.isin(["Polygon", "MultiPolygon"])]
    # Clip to city boundary (this is the key step!)
    #gdf_osm_clipped = gdf_osm.clip(city_geom)
    #only get polygons
    return gdf_osm

In [ ]:
city = 'Linn County'
state = 'Iowa'

city_geom = get_city_boundary('Cedar Rapids', 'Iowa')
center = city_geom.geometry.iloc[0].centroid
center = [center.y, center.x]  

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# Add OSM polygons (gray)
gdf_osm = load_osm_uncertain(city, state)
folium.GeoJson(
    gdf_osm,
    name="OSM Parks",
    style_function=lambda x: {"color": "red", "weight": 1, "fillOpacity": 0.1},
    tooltip=folium.GeoJsonTooltip(fields=[c for c in ["name", "leisure", "landuse", "osm_category", "highway", "information"] if c in gdf_osm.columns])
).add_to(m)

# Add layer control (toggle layers)
folium.LayerControl().add_to(m)

m.save(f"{city}_indoor.html")
m

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import pandas as pd
import glob

# Path to your unified geopackages
gpkg_files = glob.glob("../unified_geopackages_/state_11_unified.gpkg")

# Target CRS for merging
target_crs = "EPSG:4326"   # choose EPSG:5070 if you prefer equal-area

gdfs = []
for gpkg in gpkg_files:
    try:
        gdf = gpd.read_file(gpkg, layer="unified_park_area")

        # Skip if no CRS
        if gdf.crs is None:
            print(f"⚠️ {gpkg} has no CRS — skipping")
            continue

        # Harmonize CRS before concat
        gdf = gdf.to_crs(target_crs)

        gdfs.append(gdf)
    except Exception as e:
        print(f"Failed to load {gpkg}: {e}")

# Combine all into one GeoDataFrame
usa_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=target_crs)

# Optional: filter to lower 48
exclude_fips = ['02', '15', '72', '78', '66', '69', '60']
if 'state_fips' in usa_gdf.columns:
    usa_gdf = usa_gdf[~usa_gdf['state_fips'].isin(exclude_fips)]

# Reproject to Web Mercator for basemap
usa_gdf = usa_gdf.to_crs(epsg=3857)

# Plot setup
fig, ax = plt.subplots(figsize=(14, 10))
usa_gdf.plot(ax=ax, color="forestgreen", alpha=1, linewidth=0)

# Add basemap
ctx.add_basemap(
    ax,
    source=ctx.providers.CartoDB.Positron,
    alpha=0.8,
    crs=usa_gdf.crs
)

ax.set_title("Unified ParkServe + OSM Park Areas — Lower 48 States", fontsize=16)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [ ]:
usa_gdf = usa_gdf.to_crs(5070)

In [ ]:
#new area
print(usa_gdf[["city_name", "area_m2"]].head())
print("Total area:", usa_gdf["area_m2"].sum())

In [ ]:
usa_gdf["area_m2"] = usa_gdf.geometry.area

In [ ]:
print(usa_gdf[["city_name", "area_m2"]].head())
print("Total area:", usa_gdf["area_m2"].sum())

In [ ]:
import os
print("Current working directory:", os.getcwd())


In [ ]:
# Drop missing or invalid geometries
usa_gdf = usa_gdf[~usa_gdf.geometry.is_empty & usa_gdf.geometry.notna()].copy()
usa_gdf = usa_gdf[usa_gdf.geometry.is_valid].copy()

# Optional: ensure polygons only
usa_gdf = usa_gdf[usa_gdf.geometry.geom_type.isin(["Polygon", "MultiPolygon"])].copy()


In [ ]:
# Path to your unified geopackages
gpkg_files = glob.glob("../unified_geopackages_/state_01_unified.gpkg")

# Load all GeoPackages
gdfs = []
for gpkg in gpkg_files:
    try:
        gdf = gpd.read_file(gpkg, layer="unified_park_area")
        gdfs.append(gdf)
    except Exception as e:
        print(f"Failed to load {gpkg}: {e}")

# Combine all into one GeoDataFrame
usa_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Optional: filter to lower 48 (assuming you have a 'state_fips' or 'state_name' column)
# If not, skip this part — it’s only for visualization clarity
exclude_fips = ['02', '15', '72', '78', '66', '69', '60']  # AK, HI, PR, VI, GU, MP, AS
if 'state_fips' in usa_gdf.columns:
    usa_gdf = usa_gdf[~usa_gdf['state_fips'].isin(exclude_fips)]

# Reproject to Web Mercator for basemap
usa_gdf = usa_gdf.to_crs(epsg=3857)

# Plot setup
fig, ax = plt.subplots(figsize=(14, 10))
usa_gdf.plot(ax=ax, color="forestgreen", alpha=1, linewidth=0)

# Add basemap
ctx.add_basemap(
    ax,
    source=ctx.providers.CartoDB.Positron,  # or ctx.providers.Stamen.TerrainBackground
    alpha=0.8,
    crs=usa_gdf.crs
)

# Clean up the map
ax.set_title("Unified ParkServe + OSM Park Areas — Lower 48 States", fontsize=16)
ax.set_axis_off()
plt.tight_layout()
plt.show()


In [ ]:
# Path to your unified geopackages
import folium
gpkg_files = glob.glob("../unified_geopackages_updated/state_*_unified.gpkg")

# Load all GeoPackages
gdfs = []
for gpkg in gpkg_files:
    try:
        gdf = gpd.read_file(gpkg, layer="unified_park_area")
        gdf = gdf.set_crs("EPSG:4326", inplace = True,allow_override=True)
        gdfs.append(gdf)
    except Exception as e:
        print(f"Failed to load {gpkg}: {e}")

# Combine all into one GeoDataFrame

usa_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Optional: filter to lower 48 (assuming you have a 'state_fips' or 'state_name' column)
# If not, skip this part — it’s only for visualization clarity
exclude_fips = ['02', '15', '72', '78', '66', '69', '60']  # AK, HI, PR, VI, GU, MP, AS
if 'state_fips' in usa_gdf.columns:
    usa_gdf = usa_gdf[~usa_gdf['state_fips'].isin(exclude_fips)]
group1_fips = {"30", "53", "41", "38", "16"}  # MT, WA, OR, ND, ID
group2_fips = {"56", "49", "08", "06", "32", "04", "35"}  # WY, UT, CO, CA, NV, AZ, NM

selected_fips = group1_fips | group2_fips

usa_gdf_sel = usa_gdf[usa_gdf["state_fips"].astype(str).isin(selected_fips)]
print("Selected states subset:", usa_gdf_sel.shape)

center = [usa_gdf_sel.geometry.union_all().centroid.y,
          usa_gdf_sel.geometry.union_all().centroid.x]
# Reproject to Web Mercator for basemap
usa_gdf_sel = usa_gdf_sel.to_crs(epsg=3857)

m = folium.Map(location=center, zoom_start=6, tiles="cartodbpositron")

# -----------------------------
# 3. Add polygons to Folium
# -----------------------------
folium.GeoJson(
    usa_gdf_sel,
    name="Unified Park Area",
    style_function=lambda x: {
        "color": "green",
        "weight": 1,
        "fillColor": "green",
        "fillOpacity": 0.4
    }
).add_to(m)

folium.LayerControl().add_to(m)

In [ ]:
m

In [ ]:
usa_gdf.area_m2_parks.sum()

In [ ]:
usa_gdf_sel.columns

In [ ]:
(usa_gdf_sel.area_m2_parks.sum() * 10.78) / 20000000

In [46]:
#Confirming geopackage correctness through a re-run comparison.

import geopandas as gpd
import pandas as pd

gpkg_a = "../unified_geopackages_/state_17_unified.gpkg"
gpkg_b = "../unified_geopackages_just_unified/state_17_unified.gpkg"
layer = "unified_park_area"

cols = ["city_name", "area_m2_parks", "geometry"]  # include geometry
gdf_a = gpd.read_file(gpkg_a, layer=layer)[cols]
gdf_b = gpd.read_file(gpkg_b, layer=layer)[cols]





In [47]:
def area_by_city(df):
    return (
        df.groupby("city_name", dropna=False)["area_m2_parks"]
        .sum()
        .reset_index()
    )

area_a = area_by_city(gdf_a).rename(
    columns={"area_m2_parks": "area_m2_orig"}
)

area_b = area_by_city(gdf_b).rename(
    columns={"area_m2_parks": "area_m2_retry"}
)
comparison = area_a.merge(
    area_b,
    on="city_name",
    how="outer",
    indicator=True
)

comparison[["area_m2_orig", "area_m2_retry"]] = (
    comparison[["area_m2_orig", "area_m2_retry"]].fillna(0)
)

comparison["diff_m2"] = (
    comparison["area_m2_retry"] - comparison["area_m2_orig"]
)

comparison["diff_pct"] = (
    comparison["diff_m2"] / comparison["area_m2_orig"]
).replace([float("inf"), -float("inf")], None) * 100


In [48]:
AREA_EPS = 1e-6  # effectively zero

diff_cities = comparison[
    comparison["diff_m2"].abs() > AREA_EPS
].sort_values("diff_m2", ascending=False)

diff_cities


,city_name,area_m2_orig,area_m2_retry,_merge,diff_m2,diff_pct
931,New Lenox,1.649517e+06,1.962674e+06,both,313156.893367,18.98476
816,Martinsville,5.355622e+03,3.101997e+05,both,304844.078157,5692.03834
1135,Rockford,8.605390e+06,8.907120e+06,both,301729.739568,3.506288
247,Chicago,4.563341e+07,4.577985e+07,both,146440.410301,0.320906
390,Eldorado,7.739110e+04,1.634976e+05,both,86106.508996,111.261507
...,...,...,...,...,...,...
378,East Peoria,3.637384e+06,3.636652e+06,both,-732.093091,-0.020127
1119,Riverdale,2.286778e+05,2.278466e+05,both,-831.190876,-0.363477
1425,Wilmette,8.499378e+05,8.480624e+05,both,-1875.410187,-0.220653
732,Lemont,1.164877e+06,1.162271e+06,both,-2606.483962,-0.223756


In [49]:
import geopandas as gpd

# Check CRS
print(gdf_a.crs, gdf_b.crs)

# Check total area in km²
print(gdf_a['geometry'].area.sum() / 1e6)
print(gdf_b['geometry'].area.sum() / 1e6)



EPSG:5070 EPSG:4326
526.847285234074
5.6799683939812966e-08


/var/folders/qx/lhjn1vkn095_xhkfhcmh5n140000gn/T/ipykernel_16342/3784969767.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(gdf_b['geometry'].area.sum() / 1e6)


In [52]:
import geopandas as gpd
import folium

# Filter for the city
city_name = "New Lenox"
gdf_a_city = gdf_a[gdf_a['city_name'].str.lower() == city_name.lower()]
gdf_b_city = gdf_b[gdf_b['city_name'].str.lower() == city_name.lower()]

# Ensure both have geometries
if gdf_a_city.empty or gdf_b_city.empty:
    raise ValueError(f"No geometries found for {city_name} in one of the datasets.")

# Convert to WGS84 for Folium
gdf_a_city_wgs = gdf_a_city.to_crs(epsg=4326)
gdf_b_city_wgs = gdf_b_city.to_crs(epsg=4326)

# Compute a difference layer
diff_geom = gpd.overlay(gdf_b_city_wgs, gdf_a_city_wgs, how='difference')

# Center the map on the city centroid
centroid = gdf_a_city_wgs.geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=13)

# Add original geometry
folium.GeoJson(
    gdf_a_city_wgs,
    name="Original Area",
    style_function=lambda x: {"fillColor": "blue", "color": "blue", "fillOpacity": 0.3, "weight":1}
).add_to(m)

# Add retry geometry
folium.GeoJson(
    gdf_b_city_wgs,
    name="Retry Area",
    style_function=lambda x: {"fillColor": "green", "color": "green", "fillOpacity": 0.3, "weight":1}
).add_to(m)

# Add difference geometry
if not diff_geom.empty:
    folium.GeoJson(
        diff_geom,
        name="Difference",
        style_function=lambda x: {"fillColor": "red", "color": "red", "fillOpacity": 0.5, "weight":1}
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display map
m


/var/folders/qx/lhjn1vkn095_xhkfhcmh5n140000gn/T/ipykernel_16342/1115820395.py:21: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf_a_city_wgs.geometry.unary_union.centroid
